In [363]:
import pandas as pd
import numpy as np
import random

#svm
#from sklearn import svm
import tensorflow as tf
from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets("MNIST_data/", one_hot=True)

Extracting MNIST_data/train-images-idx3-ubyte.gz
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz


In [364]:
#df = pd.read_csv('../data/example.csv')
df = pd.read_csv('./data/train.csv')
df_test = pd.read_csv('./data/test.csv')

In [365]:
print("Total N Check", len(df))
print("Pclass", df.Pclass.isnull().sum())
print("Name", df.Name.isnull().sum())
print("Sex", df.Sex.isnull().sum())
print("Age", df.Age.isnull().sum())
print("SibSp", df.SibSp.isnull().sum())
print("Parch", df.Parch.isnull().sum())
print("Ticket", df.Ticket.isnull().sum())
print("Fare", df.Fare.isnull().sum())
print("Cabin", df.Cabin.isnull().sum())

Total N Check 891
Pclass 0
Name 0
Sex 0
Age 177
SibSp 0
Parch 0
Ticket 0
Fare 0
Cabin 687


In [366]:
#Sex Categorize
df = df.replace('male', 1).replace('female', 0)
#Embarked Categorize
df.replace('male', 1).replace('female', 0)
#categorize
df = df.replace(['S', 'C', 'Q'], [1, 2, 3])
#embarked 없는 사람들은 제거
df = df[df.Embarked.notnull()]
#Age가 없는사람들은 제거
df = df[df.Age.notnull()]

In [367]:
#Cabin가 없는사람들은 제거 #TBD
#df = df[df.Cabin.notnull()]

In [368]:
#unique check
print("Total N Check", len(df))
print("Pclass", df.Pclass.unique())
#print("Name", df.Name.unique())
print("Sex", df.Sex.unique())
print("Age", df.Age.unique())
print("SibSp", df.SibSp.unique())
print("Parch", df.Parch.unique())
print("Ticket", df.Ticket.unique())
print("Fare", df.Fare.unique())
print("Cabin", df.Cabin.unique())

Total N Check 712
Pclass [3 1 2]
Sex [1 0]
Age [ 22.    38.    26.    35.    54.     2.    27.    14.     4.    58.    20.
  39.    55.    31.    34.    15.    28.     8.    19.    40.    66.    42.
  21.    18.     3.     7.    49.    29.    65.    28.5    5.    11.    45.
  17.    32.    16.    25.     0.83  30.    33.    23.    24.    46.    59.
  71.    37.    47.    14.5   70.5   32.5   12.     9.    36.5   51.    55.5
  40.5   44.     1.    61.    56.    50.    36.    45.5   20.5   62.    41.
  52.    63.    23.5    0.92  43.    60.    10.    64.    13.    48.     0.75
  53.    57.    80.    70.    24.5    6.     0.67  30.5    0.42  34.5   74.  ]
SibSp [1 0 3 4 2 5]
Parch [0 1 2 5 3 4 6]
Ticket ['A/5 21171' 'PC 17599' 'STON/O2. 3101282' '113803' '373450' '17463'
 '349909' '347742' '237736' 'PP 9549' '113783' 'A/5. 2151' '347082'
 '350406' '248706' '382652' '345763' '239865' '248698' '330923' '113788'
 '347077' '19950' 'PC 17601' 'C.A. 24579' 'PC 17604' '113789' 'A./5. 2152'
 '345

In [369]:
df_selected = df[['Pclass', 'Sex', 'SibSp', 'Parch', 'Fare', 'Embarked'] ]

batch_xs = df_selected.values.tolist()

batch_ys = df['Survived'].tolist()
batch_ys_onehot = list()
for i in range(0, len(batch_ys)):
    if batch_ys[i] == 0:
        batch_ys_onehot.append([1, 0])
    else:
        batch_ys_onehot.append([0, 1])

In [370]:
Y_correct = tf.placeholder(tf.float32, shape=[None, 2])

In [371]:
#1-layer
#X = tf.placeholder(tf.float32, shape=[None, 6])
#W = tf.Variable(tf.zeros([6, 2]))
#b = tf.Variable(tf.zeros([2]))
#Y_train_predicted = tf.nn.softmax(tf.matmul(X, W) + b)
#cross_entropy = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels=Y_correct, logits=Y_train_predicted))
#learning_rate = 1e-2

In [409]:
#2-layer
X = tf.placeholder(tf.float32, shape=[None, 6])
W1 = tf.Variable(tf.random_normal([6, 4]))#tf.random_normal
b1 = tf.Variable(tf.random_normal([4]))
W2 = tf.Variable(tf.random_normal([4, 2]))
b2 = tf.Variable(tf.random_normal([2]))
Y_fc1 = tf.nn.softmax(tf.matmul(X, W1) + b1)
Y_train_predicted = tf.nn.softmax(tf.matmul(Y_fc1, W2) + b2)
cross_entropy = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels=Y_correct, logits=Y_train_predicted))
learning_rate = 1e-1

In [410]:
train_step = tf.train.GradientDescentOptimizer(learning_rate).minimize(cross_entropy)

sess = tf.InteractiveSession()
sess.run(tf.global_variables_initializer())

correct_prediction = tf.equal(tf.argmax(Y_train_predicted,1), tf.argmax(Y_correct,1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))

In [ ]:
#why random selection?
#1e-2, 1000, 500, 0.69
#1e-2, 1000, 50, 0.73
#1e-2, 5000, 50, 0.7879
#1e-2, 10000, 500, 0.7977
#1e-2, 10000, 600, 0.7935

for i in range(40000):
    batch_xs_sel = list()
    batch_ys_sel = list()
    for j in range(200): 
        random_index = random.randrange(0, len(batch_xs))
        batch_xs_sel.append(batch_xs[random_index])
        batch_ys_sel.append(batch_ys_onehot[random_index])
        
    sess.run(train_step, feed_dict={X: batch_xs_sel, Y_correct: batch_ys_sel})
    if i % 1000 == 0:
        print(accuracy.eval(feed_dict={X: batch_xs, Y_correct: batch_ys_onehot}))
    #print(b.eval())
    
    #visualize it!
    
    #tocsv file !

0.792135
0.801966
0.796348
0.801966
0.707865


In [240]:
#Sex Categorize
df_test = df_test.replace('male', 1).replace('female', 0)
#Embarked Categorize
df_test.replace('male', 1).replace('female', 0)
#categorize
df_test = df_test.replace(['S', 'C', 'Q'], [1, 2, 3])
#embarked 없는 사람들은 제거
#df_test = df_test[df_test.Embarked.notnull()]
##Age가 없는사람들은 제거
#df_test = df_test[df_test.Age.notnull()]
#Fare nan process
#np.where(df_test['Fare'].isnull())[0]
df_test['Fare'].iloc[152] = 0

C:\Anaconda3\envs\tensorflow35\lib\site-packages\pandas\core\indexing.py:141: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._setitem_with_indexer(indexer, value)


In [241]:
#nan check
df_test_selected.isnull().sum()

Pclass      0
Sex         0
SibSp       0
Parch       0
Fare        0
Embarked    0
dtype: int64

In [228]:
df_test_selected = df_test[['Pclass', 'Sex', 'SibSp', 'Parch', 'Fare', 'Embarked'] ]

In [229]:
X_test = df_test_selected.values.tolist()

In [232]:
X_test

[[3.0, 1.0, 0.0, 0.0, 7.8292, 3.0],
 [3.0, 0.0, 1.0, 0.0, 7.0, 1.0],
 [2.0, 1.0, 0.0, 0.0, 9.6875, 3.0],
 [3.0, 1.0, 0.0, 0.0, 8.6625, 1.0],
 [3.0, 0.0, 1.0, 1.0, 12.2875, 1.0],
 [3.0, 1.0, 0.0, 0.0, 9.225, 1.0],
 [3.0, 0.0, 0.0, 0.0, 7.6292, 3.0],
 [2.0, 1.0, 1.0, 1.0, 29.0, 1.0],
 [3.0, 0.0, 0.0, 0.0, 7.2292, 2.0],
 [3.0, 1.0, 2.0, 0.0, 24.15, 1.0],
 [3.0, 1.0, 0.0, 0.0, 7.8958, 1.0],
 [1.0, 1.0, 0.0, 0.0, 26.0, 1.0],
 [1.0, 0.0, 1.0, 0.0, 82.2667, 1.0],
 [2.0, 1.0, 1.0, 0.0, 26.0, 1.0],
 [1.0, 0.0, 1.0, 0.0, 61.175, 1.0],
 [2.0, 0.0, 1.0, 0.0, 27.7208, 2.0],
 [2.0, 1.0, 0.0, 0.0, 12.35, 3.0],
 [3.0, 1.0, 0.0, 0.0, 7.225, 2.0],
 [3.0, 0.0, 1.0, 0.0, 7.925, 1.0],
 [3.0, 0.0, 0.0, 0.0, 7.225, 2.0],
 [1.0, 1.0, 1.0, 0.0, 59.4, 2.0],
 [3.0, 1.0, 0.0, 1.0, 3.1708, 1.0],
 [1.0, 0.0, 0.0, 0.0, 31.6833, 1.0],
 [1.0, 1.0, 0.0, 1.0, 61.3792, 2.0],
 [1.0, 0.0, 1.0, 3.0, 262.375, 2.0],
 [3.0, 1.0, 1.0, 0.0, 14.5, 1.0],
 [1.0, 0.0, 0.0, 1.0, 61.9792, 2.0],
 [3.0, 1.0, 0.0, 0.0, 7.225, 2.0],
 [1.0

In [233]:
print("Total N Check", len(df_test))
print("Pclass", df_test.Pclass.unique())
#print("Name", df_test.Name.unique())
print("Sex", df_test.Sex.unique())
print("Age", df_test.Age.unique())
print("SibSp", df_test.SibSp.unique())
print("Parch", df_test.Parch.unique())
#print("Ticket", df_test.Ticket.unique())
print("Fare", df_test.Fare.unique())
#print("Cabin", df_test.Cabin.unique())

Total N Check 418
Pclass [3 2 1]
Sex [1 0]
Age [ 34.5   47.    62.    27.    22.    14.    30.    26.    18.    21.      nan
  46.    23.    63.    24.    35.    45.    55.     9.    48.    50.    22.5
  41.    33.    18.5   25.    39.    60.    36.    20.    28.    10.    17.
  32.    13.    31.    29.    28.5   32.5    6.    67.    49.     2.    76.
  43.    16.     1.    12.    42.    53.    26.5   40.    61.    60.5    7.
  15.    54.    64.    37.    34.    11.5    8.     0.33  38.    57.    40.5
   0.92  19.    36.5    0.75   0.83  58.     0.17  59.    14.5   44.     5.
  51.     3.    38.5 ]
SibSp [0 1 2 3 4 5 8]
Parch [0 1 3 2 4 6 5 9]
Fare [   7.8292    7.        9.6875    8.6625   12.2875    9.225     7.6292
   29.        7.2292   24.15      7.8958   26.       82.2667   61.175
   27.7208   12.35      7.225     7.925    59.4       3.1708   31.6833
   61.3792  262.375    14.5      61.9792   30.5      21.6792   31.5
   20.575    23.45     57.75      8.05      9.5      56.4958   

In [234]:
#Y_test_predicted = clf.predict(X_test)

In [237]:
Y_test_predicted

array([0, 1, 0, 0, 1, 0, 1, 0, 1, 0, 0, 0, 1, 0, 1, 1, 0, 0, 1, 1, 0, 0, 1,
       0, 1, 0, 1, 0, 0, 0, 0, 0, 1, 1, 0, 0, 1, 1, 0, 0, 0, 0, 0, 1, 1, 0,
       0, 0, 1, 1, 0, 0, 1, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 1, 1, 0, 0,
       1, 1, 0, 1, 0, 1, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 1, 1, 1, 0, 1, 0,
       1, 0, 0, 0, 1, 0, 1, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1,
       0, 0, 1, 0, 1, 1, 0, 1, 0, 0, 1, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0,
       1, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 1, 1, 0, 1, 1,
       0, 1, 0, 0, 1, 0, 0, 1, 1, 0, 0, 0, 0, 0, 1, 1, 0, 1, 1, 0, 0, 1, 0,
       1, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 1, 0, 0, 1, 0, 0, 1,
       0, 1, 0, 0, 0, 0, 1, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 1, 0, 1, 0, 0,
       0, 1, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 0, 0, 0, 0, 1, 0, 1, 1, 1, 0, 0,
       0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 1, 0, 0, 0, 0, 1, 0, 0, 0, 1, 1, 0, 1,
       0, 0, 0, 0, 1, 0, 1, 1, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0,
       0, 0,

In [238]:
f = open('answer.csv','w')
#header
f.write('PassengerId,Survived\n') 
for i in range(0, len(df_test)):
    line = str(df_test['PassengerId'].iloc(i)[i]) + ',' + str(Y_test_predicted[i]) + '\n'
    f.write(line) 
f.close()